In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import nltk
from nltk.corpus import stopwords
import re
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, BertConfig


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
train_path="/content/drive/MyDrive/Colab Notebooks/train (1).csv"
df = pd.read_csv(train_path)

In [4]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


**Clean Text**

In [6]:
nltk.download('stopwords')
sw = stopwords.words('english')

def clean_text(text):

    text = text.lower()

    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs
    #text = re.sub(r"http", "",text)

    html=re.compile(r'<.*?>')

    text = html.sub(r'',text) #Removing html tags

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations

    text = [word.lower() for word in text.split() if word.lower() not in sw]

    text = " ".join(text) #removing stopwords

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis

    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
df['text'] = df['text'].apply(lambda x: clean_text(x))

In [8]:
tweets = df.text.values
labels = df.target.values


**BERT Tokenizer**

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
print('Original: ', tweets[0])
print('Tokenized: ', tokenizer.tokenize(tweets[0]))
print('Token IDs: ',tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets[0])))

Original:  deeds reason earthquake may allah forgive us
Tokenized:  ['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us']
Token IDs:  [15616, 3114, 8372, 2089, 16455, 9641, 2149]


In [11]:
max_len = 0

for sent in tweets :
  input_ids = tokenizer.encode(sent, add_special_tokens = True)

  max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len )

Max sentence length:  45


In [12]:
input_ids = []
attention_masks=[]

for tweet in tweets:
  encoded_dict = tokenizer.encode_plus(tweet, add_special_tokens = True, max_length = max_len,pad_to_max_length= True,return_tensors='pt')
  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])
#chuyển dãy về dạng tesor
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

print("Original: ", tweets[0])
print("Token IDS: ",input_ids[0])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  deeds reason earthquake may allah forgive us
Token IDS:  tensor([  101, 15616,  3114,  8372,  2089, 16455,  9641,  2149,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])


In [13]:
dataset = TensorDataset(input_ids, attention_masks,labels)

train_size = int(0.8*len(dataset))

val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size,val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

6,090 training samples
1,523 validation samples


In [14]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)

validation_dataloader = DataLoader(val_dataset, sampler = RandomSampler(val_dataset),batch_size = batch_size)


#**Initializing BERT**


In [15]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = False, output_hidden_states = False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
for param in model.parameters():
  param.requires_grad = False

#**Add LoRA layers**

In [17]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
class LoRALayer(nn.Module):
  def __init__(self,in_dim, out_dim, rank, alpha):
    super().__init__()
    std_dev = 1/torch.sqrt(torch.tensor(rank).float())
    self.W_a = nn.Parameter(torch.randn(in_dim, rank)*std_dev)
    self.W_b = nn.Parameter(torch.zeros(rank, out_dim))
    self.alpha = alpha
  def forward(self,x):
    x = self.alpha * (x @ self.W_a@ self.W_b)
    return x

class LinearWithLoRA(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    self.linear = linear
    self.lora = LoRALayer(linear.in_features, linear.out_features, rank, alpha)
  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [23]:
from functools import partial

lora_r = 8
lora_alpha = 16

lora_dropout = 0.05
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False

layers = []

assign_lora = partial(LinearWithLoRA, rank = lora_r, alpha = lora_alpha)

for layer in model.bert.encoder.layer :
  if lora_query:
    layer.attention.self.query = assign_lora(layer.attention.self.query)
  if lora_key:
    layer.attention.self.key = assign_lora(layer.attention.self.key)
  if lora_value:
    layer.attention.self.value = assign_lora(layer.attention.self.value)
  if lora_projection:
    layer.attention.output.dense = assign_lora(layer.attention.output.dense)
  if lora_mlp:
    layers.intermediate.dense = assign_lora(layers.intermediate.dense)
    layers.output.dense = assign_lora(layer.output.dense)




In [24]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): LinearWithLoRA(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora): LoRALayer()
              )
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): LinearWithLoRA(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora): LoRALayer()
              )
              (dropout): Dropout(p=0.1, inplac

In [25]:
# Check if linear layers are frozen
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

bert.embeddings.word_embeddings.weight: False
bert.embeddings.position_embeddings.weight: False
bert.embeddings.token_type_embeddings.weight: False
bert.embeddings.LayerNorm.weight: False
bert.embeddings.LayerNorm.bias: False
bert.encoder.layer.0.attention.self.query.linear.weight: False
bert.encoder.layer.0.attention.self.query.linear.bias: False
bert.encoder.layer.0.attention.self.query.lora.W_a: True
bert.encoder.layer.0.attention.self.query.lora.W_b: True
bert.encoder.layer.0.attention.self.key.weight: False
bert.encoder.layer.0.attention.self.key.bias: False
bert.encoder.layer.0.attention.self.value.linear.weight: False
bert.encoder.layer.0.attention.self.value.linear.bias: False
bert.encoder.layer.0.attention.self.value.lora.W_a: True
bert.encoder.layer.0.attention.self.value.lora.W_b: True
bert.encoder.layer.0.attention.output.dense.weight: False
bert.encoder.layer.0.attention.output.dense.bias: False
bert.encoder.layer.0.attention.output.LayerNorm.weight: False
bert.encoder.lay

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 294912


#**Finetuning**

In [27]:
epochs = 4
total_steps = len(train_dataloader) * epochs
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps= 0 , num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
#Tính accuracy
def cal_accuracy(preds, labels) :
  pred_flat = np.argmax(preds, axis = 1 ).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat)/len(labels_flat)

In [29]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [31]:
model = model.to(device)
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []
#thoi gian training
total_t0 = time.time()

for epoch_i in range(0,epochs):
  #training
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training....')

  #tinh thoi gian train cho moi epoch
  t0 = time.time()
  total_train_loss = 0
  model.train()
  for step, batch in enumerate(train_dataloader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    optimizer.zero_grad()
    output = model(b_input_ids, token_type_ids = None, attention_mask= b_input_mask, labels = b_labels)
    loss = output.loss
    total_train_loss += loss.item()
    #backward de tinh gradient
    loss.backward()
    #chuan hoa gradient ve khoang 0 1 de tranh "exploding gradients"
    torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
    #cap nhat tham so mo hinh
    optimizer.step()
    #cap nhat learning rate
    scheduler.step()

  #tinh trung binh loi cua tat ca cac batch
  avg_train_loss = total_train_loss/ len(train_dataloader)

  #tinh thoi gian train
  training_time = format_time(time.time()-t0)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epcoh took: {:}".format(training_time))
  #Validation
  print("")
  print("Running Validation...")
  t0= time.time()
  model.eval()
  total_eval_accuracy = 0
  best_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0
  for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():
        output= model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=b_labels)
    loss = output.loss
    total_eval_loss += loss.item()
    # Move logits and labels to CPU if we are using GPU
    logits = output.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += cal_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
  avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
  print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
  avg_val_loss = total_eval_loss / len(validation_dataloader)
  # Measure how long the validation run took.
  validation_time = format_time(time.time() - t0)
  if avg_val_accuracy > best_eval_accuracy:
    torch.save(model, 'bert_model')
    best_eval_accuracy = avg_val_accuracy
  training_stats.append(
    {
      'epoch': epoch_i + 1,
      'Training Loss': avg_train_loss,
      'Valid. Loss': avg_val_loss,
      'Valid. Accur.': avg_val_accuracy,
      'Training Time': training_time,
      'Validation Time': validation_time
    }
  )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))





======== Epoch 1 / 4 ========
Training....

  Average training loss: 0.52
  Training epcoh took: 0:00:33

Running Validation...
  Accuracy: 0.81

======== Epoch 2 / 4 ========
Training....

  Average training loss: 0.44
  Training epcoh took: 0:00:32

Running Validation...
  Accuracy: 0.80

======== Epoch 3 / 4 ========
Training....

  Average training loss: 0.39
  Training epcoh took: 0:00:35

Running Validation...
  Accuracy: 0.80

======== Epoch 4 / 4 ========
Training....

  Average training loss: 0.37
  Training epcoh took: 0:00:33

Running Validation...
  Accuracy: 0.82

Training complete!
Total training took 0:02:38 (h:mm:ss)


In [32]:
model = torch.load('bert_model')

In [33]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
df_test['text'] = df_test['text'].apply(lambda x:clean_text(x))
test_tweets = df_test['text'].values

In [34]:
test_input_ids = []
test_attention_masks = []
for tweet in test_tweets:
    encoded_dict = tokenizer.encode_plus(
                        tweet,
                        add_special_tokens = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [35]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [36]:
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()

            predictions.extend(list(pred_flat))